# HPE OneView API Workshop
## Introduction to the HPE OneView API
To facilitate the usage of the API from various programming languages and to avoid creating code to build HTTP requests and all their attributes (headers, body), HPE offers libraries for several languages that take care of REST details allowing you to concentrate on the resources you want to manage. In this workshop we are going to use PowerShell. Version 5.0 (and later) of the HPE OneView PowerShell library is compatible with [PowerShell Core](https://github.com/PowerShell/PowerShell). It can work on different platforms, including Windows of course, but also on macOS and Linux.  

To start, we first need to load our HPE OneView PowerShell library 

In [ ]:
Import-Module HPOneView.520

You can get a list of all the commands provided by the library:

In [ ]:
Get-Command -Module HPOneView.520

Let's define the IP address of the HPE OneView appliance we want to connect to.

In [ ]:
$appliance = "$$OVIP"

Now you can connect to the HPE OneView appliance

In [ ]:
Connect-HPOVMgmt -hostname $appliance -username Administrator -password password

If the login is successful, you will get an authenticated session with your HPE OneView appliance. You will find the key to that session in a string stored in the following variable.

In [ ]:
$token = $global:ConnectedSessions[0].SessionID
$token

You can use this token to access the REST API without using the PowerShell library. For example, you can use curl to make a simple HTTP request, passing the session ID in a HTTP header named auth

In [ ]:
curl -k "https://$appliance/rest/network-sets" -H "auth: $token" | jq .

Try the same request without the auth header, and see what error you get

In [ ]:
curl -k "https://$appliance/rest/network-sets" | jq .

This same request can be done with a PowerShell cmdlet from the library. This will take care of the URL for the request (https://.../rest/network-sets), the auth header and other headers like X-API-Version. If you want, you can add the `-Verbose` option to see a lot more details about the command, including HTTP URLs and the headers of the request and response

In [ ]:
Get-HPOVNetworkSet

Let's look at the impact of another important header: X-API-Version. This header defines the version of the API endpoint your code was designed for. Each version of HPE OneView has a corresponding API version, and no they are not the same. The mapping can be viewed in the first page of the online help for HPE OneView entitled About: https://techlibrary.hpe.com/docs/enterprise/servers/oneview5.2/cicf-api/en/index.html#about  

For backwards compatibility, every version of HPE OneView accepts requests for its own API version and several older versions. The list of supported and unsupported API versions can also be found in that help page above. It is important to understand that without an X-API-Version header in the request, HPE OneView will assume you want the oldest version of the API it supports. 

Let's consider an example with a feature that was added relatively recently in HPE OneView, hypervisor cluster profiles. What happens if you try a GET on that resource without an X-API-Version header ?

In [ ]:
curl -k "https://$appliance/rest/hypervisor-cluster-profiles" -H "auth: $token" | jq .

You will get a 404 error, the well-known page not found. That is simply because in older versions of the API, this resource did not exist. Now repeat the same request with an X-API-Version header

In [ ]:
curl -k "https://$appliance/rest/hypervisor-cluster-profiles" -H "auth: $token" -H "X-API-Version: 1600" | jq .

Omitting the X-API-Version header is one of the most common mistakes users make when using the HPE OneView API without one of the helper libraries (like the PowerShell one we are using in this workshop). This can cause some head scratching because it may still work in some cases and not work or cause weird behavior in other cases. In the case of the ethernet-networks example above, it will work because this resource has existed since the earliest version of HPE OneView.

## Provision a new server

Our goal is to provision a new server to augment the capacity of a business application. We can look at the servers managed by HPE OneView and see which ones would be available

In [ ]:
Get-HPOVServer

This default output does not tell us if a server has a profile assigned to it, but it doesn't show all the data. Let's look at what properties are available in a server hardware resource

In [ ]:
Get-HPOVServer | Get-Member

We see serverProfileUri. This is the property that tells us if a server hardware has a profile assigned to it. We want a list of servers that do not have a server profile assigned, i.e. where the serverProfileUri is empty

In [ ]:
Get-HPOVServer | Where-Object {!$_.serverProfileUri}

For those of you more experienced in using HPE OneView PowerShell, there is an easier way to perform the above command, which is by using the -NoProfile option of Get-HPOVServer...

In [ ]:
Get-HPOVServer -NoProfile

It is fair to assume that any of these servers is available. We want to create a profile to assign to one of these servers. We have server profile templates that were created outside of this workshop.

In [ ]:
Get-HPOVServerProfileTemplate

You are going to use the server named $$SRVNAME which is linked to Server Hardware Type $$SHT. Now create a server profile based on server profile template $$SPTNAME

In [ ]:
$server = Get-HPOVServer -Name "$$SRVNAME"
$spt = Get-HPOVServerProfileTemplate -name "$$SPTNAME"
New-HPOVServerProfile -Name "Profile Student $$I" -ServerProfileTemplate $spt -server $server

Let's look in detail at the server profile that was just created. Most of its properties come from the Server Profile Template

In [ ]:
get-hpovserverprofile -name "Profile Student $$I" | select *

As always in PowerShell, piping is very useful. Let's determine which servers are available (no profile assigned) for our given Server Hardware Type $$SHT

In [ ]:
get-hpovserverhardwaretype -name "$$SHT" | get-hpovserver -noprofile

The PowerShell HPE OneView library provides cmdlets for just about everything in the HPE OneView REST API. In those rare cases not covered by a native cmdlet, you can use the more generic function Send-HPOVRequest and pass it the URI you want /rest/...  
Do this to get the list of active sessions on the appliance. The REST call is documented [here](https://techlibrary.hpe.com/docs/enterprise/servers/oneview5.0/cicf-api/en/index.html#rest/active-user-sessions)

In [ ]:
Send-HPOVRequest -uri "/rest/active-user-sessions"

To clean up, delete the server profile we created previously. This will allow you to re-create it for the same server in the Python notebook we are about to look at.

In [ ]:
Remove-HPOVServerProfile -Name "Profile Student $$I" -Confirm:$false

Finally, when you're done with your work with the HPE OneView API, it is good practice to disconnect from the appliance to clean up the session.

In [ ]:
Disconnect-HPOVMgmt